## Conversion of CAMBOTv2 GPS antenna positions  
## to the camera's focal plane position

The ATM CAMBOTv2 L0 data product also provides position information for each L0 raw image. To use these GPS positions in ATM auxiliary files for processing with the Ames Stereo Pipeline ([ASP](https://stereopipeline.readthedocs.io/en/latest/index.html)) they have to be converted from GPS antenna coordinates to the camera's focal plane position. This is done by rotating and translating the antenna phase center coodinate using attitude (pitch, roll, and yaw) information and the lever arm between the GPS antenna's phase center and the camera's focal plane. All information is provided in the auxilliary navigation files.

The files are part of NASA's Airborne Topographic Mapper (ATM) CAMBOTv2 L0 raw images and are freely available from the National Snow and Ice Data Center (data set ID: IOCAM0):  
https://nsidc.org/data/iocam0/versions/1  
DOI: https://doi.org/10.5067/IOJH8A5F48J5  
**NOTE:** An example auxilliary navigation file is included in this repository in the folder `data/nav` and will be used in this notebook.

The matrices for the coordinate transformation are described in the geolocation documentation:  
https://nsidc.org/sites/default/files/oib_cambot_geolocation.pdf

First, load the required modules and set up the input and output file names for conversion.

In [1]:
import pandas as pd
import numpy as np
import pyproj
import re
import os

# set input directory with navitagion file for conversion
f_dir_nav = r".." + os.sep + "data" + os.sep + "nav"

# set input and output file names
f_name_aux = f_dir_nav + os.sep + "IOCAM0_2019_GR_NASA_20190906_ancillary_data.csv"
f_name_asp = f_dir_nav + os.sep + "IOCAM0_2019_GR_NASA_20190906_camera_positions.csv"

VERBOSE = True # displays header paramters if desired

ATM auxiliary (aux) file format:

    # CAMBOT positioning and installation info for 2019_GR_NASA on GV (N95NA)
    # Input ancillary file: 20190906_CAMBOT_ancillary_precise.csv, updated: Mon Jul 13 17:20:49 2020
    # Camera offset from GPS antenna (in meters) [x-forward, y-starboard, z-down]: -4.463, 0.092, 2.042
    # Camera angular mounting biases (in deg, to be added to attitude measurements) [pitch, roll, heading]: 0.0, 0.0, 0.0
    # Range bias (in m, already added to range measurement): 0
    # Time offset (in sec, to be added to image timestamp to get UTC): -0.4338
    #         ImageFilename,             Timestamp(UTC), PosixTime(UTC),     Lat(deg),     Lon(deg), AntAlt(m),    AGL(m), Roll(deg), >Pitch(deg), Heading(deg)
    IOCAM0_2019_GR_NASA_20190906-112100.4216.jpg, 2019-09-06T11:21:00.000000, 1567768860.000,    76.493496,   -68.125623,  1193.617, >-9999.000,     5.518,      3.786,       55.536
    IOCAM0_2019_GR_NASA_20190906-112100.9217.jpg, 2019-09-06T11:21:00.500000, 1567768860.500,    76.493816,   -68.123564,  1193.840, -9999.000,     6

The ASP camera position format can be set with processing parameters: `--csv-format "1:file 2:lon 3:lat 4:height_above_datum"` in the `camera_solve` tool.

>`# ID, longitude, latitude, elevation`  
>`2009_11_05_00266, -62.73216039, -62.95223574, 10089`  
>`2009_11_05_00267, -62.71861428, -62.95946623, 10088`

In [2]:
# parse header of ATM aux file and extract all information from header
# this section of the code was written by C. Wayne Wright (https://github.com/lidar532)

def parse_cambot_header(
    file_name:str,    # CAMBOTv2 file to read and extract header info from.
    max_chars=2000,   # Maximum number of characters to read in.
    debug:bool=False  # True to get debuggin printout, False for no debug printout
    ) -> object:      # A class containing the extracted data if it was found.
  """
  Parse a CAMBOT header and extract the fwd, right, down, pitch, roll, heading, Time_Offset, and Range_Bias values.
  and return a class with those values.

  asof: 2024-0214.
  """

  # regex expressions built with: https://regex101.com/
  rx_offset       = r"\[x-forward, y-starboard, z-down\]: (?P<fwd>-*\d+\.\d*), (?P<right>-*\d+\.\d+), *(?P<down>-*\d*\.\d*)"
  rx_attitude     = r"\[pitch, roll, heading\]: (?P<pitch>-*\d+\.\d*), *(?P<roll>-*\d*\.\d*), *(?P<heading>-*\d\.\d*)"
  rx_time_offset  = r"Time offset .*: *(?P<Time_Offset>-*\d\.\d*)"
  rx_range_bias   = r"Range bias.*: *(?P<Range_Bias>-*\d*\.*\d)"
  rx_file_name    = r"Input ancillary file: *(?P<aux_file_name>.*),"
  rx_header_row   = r"# *(ImageFilename)"
  row_number      = 0

  cb_fd = open(f_name_aux, 'r')
  cb_header = cb_fd.readlines(max_chars)
  if debug:
    for line in cb_header:
      print(f"line={line}", end="")
  cb_fd.close()

  # define a class to return the data we are able to extract.
  class CB_HEADER():
    def __init__(self):
      self.header_row     = None
      self.fwd            = None
      self.right          = None
      self.down           = None
      self.pitch          = None
      self.roll           = None
      self.heading        = None
      self.Time_Offset    = None
      self.Range_Bias     = None
      self.header_row     = None
      self.data_start_row = None
      self.headers        = None

  #  instantiate the return variable "rv" as a CB_HEADER class.
  rv = CB_HEADER()

  # Walk over the lines of the header.
  for line in cb_header:
    if debug:
      print(f'{row_number:3d}: {line}', end='')
    x = re.search(rx_offset, line, flags=0)
    if x:
      rv.fwd   = float(x.group('fwd'))
      rv.right = float(x.group('right'))
      rv.down  = float(x.group('down') )
      if debug:
        print(f'        {rv.fwd = } {rv.right = } {rv.down = }')

    x = re.search(rx_attitude, line, flags=0)
    if x:
      rv.pitch   = float(x.group('pitch'))
      rv.roll    = float(x.group('roll'))
      rv.heading = float(x.group('heading'))
      if debug:
        print(f'      {rv.pitch = } {rv.roll = } {rv.heading = }')

    x = re.search(rx_time_offset, line, flags=0)
    if x:
      rv.Time_Offset = float(x.group('Time_Offset'))
      if debug:
        print(f'{rv.Time_Offset = }')

    x = re.search(rx_range_bias, line, flags=0)
    if x:
      rv.Range_Bias = float(x.group('Range_Bias'))
      if debug:
        print(f' {rv.Range_Bias = }')

    x = re.search(rx_file_name, line, flags=0)
    if x:
      rv.aux_file_name = x.group('aux_file_name')
      if debug:
        print(f' {rv.aux_file_name = }')

    x = re.search(rx_header_row, line, flags=0)
    if x:
      rv.row_number = row_number
      rv.header_row = line
      if debug:
        print(f' {row_number = }')
    row_number += 1

  if rv.header_row:
    rv.data_start_row = rv.row_number + 1
    rv.header_row = rv.header_row.strip()
    rv.headers = rv.header_row.replace("(", "_").replace(")", "").replace(" ", "").replace("#","").replace(">","").split(",")
  return rv

#%% read data

header_data = parse_cambot_header( f_name_aux , debug=False )

if VERBOSE:
    for k in header_data.__dict__:
      print(f'{k:15s}: {header_data.__dict__[k]}' )

header_row     : #          ImageFilename,             Timestamp(UTC), PosixTime(UTC),     Lat(deg),     Lon(deg), AntAlt(m),    AGL(m), Roll(deg), Pitch(deg), Heading(deg)
fwd            : -4.463
right          : 0.092
down           : 2.042
pitch          : 0.0
roll           : 0.0
heading        : 0.0
Time_Offset    : -0.4338
Range_Bias     : 0.0
data_start_row : 7
headers        : ['ImageFilename', 'Timestamp_UTC', 'PosixTime_UTC', 'Lat_deg', 'Lon_deg', 'AntAlt_m', 'AGL_m', 'Roll_deg', 'Pitch_deg', 'Heading_deg']
aux_file_name  : 20190906_CAMBOT_ancillary_precise.csv
row_number     : 6


In [3]:
# extract lever arm parameters from class if they exist
if (hasattr(header_data, 'fwd') &  hasattr(header_data, 'right') &  hasattr(header_data, 'down')):
    lever_arm_sensor = np.array([header_data.fwd, header_data.right, header_data.down])
else:
    os.sys.exit("Unable to extract lever arm parameters from ATM aux file. Check input data.")

In [4]:
# read data from ATM aux file
df = pd.read_csv(f_name_aux, skiprows=header_data.data_start_row, names=header_data.headers)
if VERBOSE:
    print(df.head())  

                                  ImageFilename                Timestamp_UTC  \
0  IOCAM0_2019_GR_NASA_20190906-112100.4216.jpg   2019-09-06T11:21:00.000000   
1  IOCAM0_2019_GR_NASA_20190906-112100.9217.jpg   2019-09-06T11:21:00.500000   
2  IOCAM0_2019_GR_NASA_20190906-112101.4217.jpg   2019-09-06T11:21:01.000000   
3  IOCAM0_2019_GR_NASA_20190906-112101.9217.jpg   2019-09-06T11:21:01.500000   
4  IOCAM0_2019_GR_NASA_20190906-112102.4216.jpg   2019-09-06T11:21:02.000000   

   PosixTime_UTC    Lat_deg    Lon_deg  AntAlt_m   AGL_m  Roll_deg  Pitch_deg  \
0   1.567769e+09  76.493496 -68.125623  1193.617 -9999.0     5.518      3.786   
1   1.567769e+09  76.493816 -68.123564  1193.840 -9999.0     6.684      3.885   
2   1.567769e+09  76.494134 -68.121499  1194.097 -9999.0     8.310      3.961   
3   1.567769e+09  76.494449 -68.119427  1194.421 -9999.0     9.786      3.876   
4   1.567769e+09  76.494761 -68.117347  1194.748 -9999.0    10.663      3.932   

   Heading_deg  
0       55.536 

In [5]:
# populate numpy arrays with needed data

ant_lon = np.asarray(df["Lon_deg"])   # float64
ant_lat = np.asarray(df["Lat_deg"])   # float64
ant_ele = np.asarray(df["AntAlt_m"])  # float64

pitch   = np.asarray(df["Pitch_deg"]) # float64
roll    = np.asarray(df["Roll_deg"])  # float64
yaw     = np.asarray(df["Heading_deg"])  # float64

agl     = np.asarray(df["AGL_m"])   # float64

# convert coordinate and attitude angles from degrees to radians and use vectorizationg for faster execution
lon_rad = np.deg2rad(ant_lon)
lat_rad = np.deg2rad(ant_lat)
p_rad = np.deg2rad(pitch)
r_rad = np.deg2rad(roll)
y_rad = np.deg2rad(yaw)

In [6]:
# set up projections
transformer_geo2ecef = pyproj.Transformer.from_crs(
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    )
transformer_ecef2geo = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )
# convert phase center geodetic positions to Earth-centered, Earth-fixed coordinates (ECEF) 
x_ant_ecef, y_ant_ecef, z_ant_ecef = transformer_geo2ecef.transform(ant_lon,ant_lat,ant_ele,radians=False)
del ant_lon, ant_lat

In [7]:
# =============================================================================
# helper function for converting antenna phase center ECEF coordinates
# to sensor ECEF coordinates using lever arm and attitude information
# =============================================================================

def ecef_ant_pos_to_sensor_pos(ant_lon,ant_lat,x_ant_ecef, y_ant_ecef, z_ant_ecef,pitch, roll, yaw, lever_arm):

    """
    SUMMARY:       The ecef_ant_pos_to_sensor_pos function calculates the location of an instrument's sensor using the geodetic coordinates of the 
                   phase center of the aircraft's GPS antenna, pitch, roll, and yaw (heading) and the instrument's lever arm measured in an
                   aircraft-fixed cartesian coordinate system. The output is in a geocentric ECEF (Earth-centered, Earth-fixed) coordinate system 

    INPUT:
        
    lon            antenna in radians    

    pitch          Aircraft pitch angle in radians from IMU data. Positive pitch = aircraft nose up. 
                   This parameter is also stored in /aircraft/pitch in ATM's waveform HDF5 files available from NSDIC (data products ILATMW1B and ILNSAW1B).

    roll           Aircraft roll angle in radians from IMU data. Positive roll = starboard (right) wing down.
                   This parameter is also stored in /aircraft/pitch in ATM's waveform HDF5 files available from NSDIC (data products ILATMW1B and ILNSAW1B).

    yaw            Aircraft heading angle in radians true north from IMU data. Positive towards east. 
                   This parameter is also stored in /aircraft/heading in ATM's waveform HDF5 files available from NSDIC (data products ILATMW1B and ILNSAW1B).

    lever_arm      3x1 displacement vector of the center of the camera's focal plane measured from the phase center
                   of the aircraft's GPS antenna towards the scan mirror in an aircraft-fixed cartesian coordinate system.
                   lever_arm = [lever_arm_x, lever_arm_y, lever_arm_z]
                   
    OUTPUT:        vector with ECEF coordinates of the camera's focal plane

    SYNTAX:        sensor_ecef = ecef_ant_pos_to_sensor_pos(ant_ecef_x, ant_ecef_y, ant_ecef_z, pitch, roll, heading,lever_arm)
    """
    
    ant_ecef = np.array([x_ant_ecef, y_ant_ecef, z_ant_ecef])
    
    # set up rotation matrix to align sensor coordinate system with aircraft coordinate system with the x-axis in the direction of North
    # the T(heading,pitch,roll) rotation aligns the sensor’s coordinate system with the aircraft coordinate system, with the x-axis in the direction of the North

    cp = np.cos(pitch)
    sp = np.sin(pitch)
    cr = np.cos(roll)
    sr = np.sin(roll)
    ch = np.cos(yaw)
    sh = np.sin(yaw)
    
    T = np.array([ [ch*cp, ch*sp*sr-sh*cr, ch*sp*cr+sh*sr],
          [sh*cp, sh*sp*sr+ch*cr, sh*sp*cr-ch*sr],
          [-1.0*sp,          cp*sr,          cp*cr ]])
    
    # rotate the sensor coordinates into a local North, East, Down (NED) coordinate system

    st = np.sin(ant_lat)
    ct = np.cos(ant_lat)
    sl = np.sin(ant_lon)
    cl = np.cos(ant_lon)
    
    NED_R = np.array([ [-1.0*st*cl, -1.0*sl, -1.0*ct*cl],
                [-1.0*st*sl ,     cl , -1.0*ct*sl],
                        [ct  ,     0  ,  -1.0*st] ])    

    sensor_ecef = NED_R @ T @ lever_arm_sensor + ant_ecef
 
    return sensor_ecef


In [8]:
# allocate empty arrays for faster execution - NOTE: calculations need to be done with float64 to get the required accuracy
sensor_ecef_x = np.empty(ant_ele.size) # float64
sensor_ecef_y = np.empty(ant_ele.size) # float64
sensor_ecef_z = np.empty(ant_ele.size) # float64

for i in range(ant_ele.size):
   sensor_ecef = ecef_ant_pos_to_sensor_pos(lon_rad[i],lat_rad[i],x_ant_ecef[i], y_ant_ecef[i], z_ant_ecef[i],p_rad[i], r_rad[i], y_rad[i], lever_arm_sensor)
   sensor_ecef_x[i] = sensor_ecef[0]
   sensor_ecef_y[i] = sensor_ecef[1]
   sensor_ecef_z[i] = sensor_ecef[2]


In [9]:
# transform sensor ECEF coordinates to geodetic
lon_sen, lat_sen, alt_sen = transformer_ecef2geo.transform(sensor_ecef_x,sensor_ecef_y,sensor_ecef_z,radians=False)

In [10]:
# export new coordinates and also include attitude information
f = open(f_name_asp, 'w')
f.write("# ID, longitude_deg, latitude_deg, elevation_m, pitch_deg, roll_deg, yaw_deg\n")
for i in range(ant_ele.size):
    f.write(f'{df.iloc[i, 0]:s},{lon_sen[i]:25.20f},{lat_sen[i]:25.20f},{alt_sen[i]:10.4f} ,{pitch[i]:10.4f}, {roll[i]:10.4f},{yaw[i]%360:10.4f}\n')
f.close()

print(f'Converted {ant_ele.size:d} phase center positions to camera focal plane positions: {f_name_asp:s}')

Converted 42691 phase center positions to camera focal plane positions: ..\data\nav\IOCAM0_2019_GR_NASA_20190906_camera_positions.csv
